## Data

For this project, I used the online wikipedia webpage (https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles) to get the neighborhoods of Los Angeles city.

Since I need to extract data from a webpage, I imported library of BeautifulSoup to clean the html file. I slicsed the data from the webpage contained the neighborhoods names and created a pandas dataframe with one column of Neighborhoods for further analysis. 

Then I imported nominatim to find the latitude and longitude of the corresponding neighborhood followed by inserting into the previous dataframe with column names of latitude and longitude. 


In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np
import re

# get the table's original data
source = requests.get('https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles').text
soup = BeautifulSoup(source,'lxml')

# get all data of the neighborhood(except ones can't find the latitude and longitude, 
# also the neighborhoods that far from the central Los Angeles) and make a list
data = []

table = soup.find_all('div', class_='div-col')

for raw_data  in  table[0].find_all('li'):
    all_data=raw_data.text
    all_data=all_data.rstrip()
    if re.search("/", all_data) or re.search(",", all_data) or re.search("Beachwood Canyon", all_data) or re.search("Holmby Hills", all_data) or re.search("NoHo Arts District", all_data) or re.search("Picfair Village", all_data) or re.search("Yucca Corridor", all_data):
        continue
    data.append(all_data)

for raw_data  in  table[1].find_all('li'):
    all_data=raw_data.text
    all_data=all_data.rstrip()
    if re.search("/", all_data) or re.search(",", all_data) or re.search("Beachwood Canyon", all_data) or re.search("Holmby Hills", all_data) or re.search("NoHo Arts District", all_data) or re.search("Picfair Village", all_data) or re.search("Yucca Corridor", all_data):
        continue
    data.append(all_data)
# creat the dataframe
data_array0=np.array(data)
data_array1 = [x.split("[")[0] for x in data_array0]

not_wanted = ["Arlington Heights","Canterbury Knolls","Del Rey", "Edendale", "Harvard Park", "Ladera", "Nichols Canyon", "Rancho Park", "Sunland","Valley Glen"]
data_array = [x.split("[")[0] for x in data_array1 if x not in not_wanted]

df = pd.DataFrame(data_array, columns ={"Neighborhood"})
df.columns = ['Neighborhood']
df['Latitude']=''
df['Longitude']=''
df.shape


(176, 3)

In [2]:
# use geopy to get the latitude and longitude of neighborhoods of Los Angeles

from geopy.geocoders import Nominatim

for x in range(len(data_array)):
    
    neighborhood_name = df['Neighborhood'][x]
    address = neighborhood_name +', CA' 
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
   
    df['Latitude'][x] = latitude
    df['Longitude'][x] = longitude

df

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.0703,-118.255
1,Arleta,34.2413,-118.432
2,Arts District,42.3178,-83.0415
3,Atwater Village,34.1164,-118.256
4,Baldwin Hills,34.0076,-118.351
5,Baldwin Village,47.5956,-57.6407
6,Baldwin Vista,34.4302,-119.732
7,Benedict Canyon,34.0494,-118.4
8,Beverly Crest,35.4645,-80.826
9,Beverly Glen,43.0554,-82.1759


Then I imported geopy library and foium library for making a map. There are totally 176 neighborhoods in this dataframe. I created a map of neighborhoods of Los Angeles city.

After cleaning, selecting and visualizing the dataset, I utilized Foursquare API to explore the neighborhoods of Los Angeles. Foursquare API has a massive dataset of location data. They crowdsourced their data and then we can build our dataset and add venues and explore the region that we are interested in. In my project, I requested the top 100 venus in Los Angeles region with four columns of venus name, venus catogories, venus latitude and longitude respectly. 

Before analyzing the dataset, I need to create a new dataframe including neighborhood name, neighborhood latitude, neighborhood longtitude, venus name, venus latitude, venus longtitude and venus categories. Then I inputed each time the venus was visited. Now I can group rows by neighborhood name and take the mean of the frequency of occurrence of each venus category. Then the dataframes of the top ten most common venus were created. 

After I customerize the dataset, I can apply customer segmentation on this historical data. The algorithms that I used for segmentation is K-means clustering. The number of cluster that I choosed is 5. I imported Kmeans to cluster the dataset and imported forlium to visualize the five  clusters.